# Double peak area
read the csv file of double peaked fields, and compute their area

In [59]:
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
from IPython.display import Image
from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy

import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

In [3]:
data_dir_base = "/Users/hn/Documents/01_research_data/remote_sensing/02_peaks_and_plots/"
param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

# Define Parameters

In [37]:
double_crop_potential_plants = pd.read_csv(param_dir + "double_crop_potential_plants.csv")
double_crop_potential_plants.head(2)

,Crop_Type,Category
0,Artichoke,Annual
1,Barley,Annual


In [6]:
county = "Grant"
year = 2016

In [65]:
#
#  function parameters
#
look_ahead = 8
freedom_dg = 9
data_dir = data_dir_base + county + "_" + str(year) + "/"

# Read data

In [66]:
double_dt = pd.read_csv(data_dir + "LA_" + str(look_ahead) + "_df_" + str(freedom_dg) + "_double_polygons.csv")
double_dt.tail(2)

,Acres,CovrCrp,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,county,year,geo
8,88.0,NaN,Hay/Silage,Alfalfa Hay,NaN,88.395109,2003/07/01 00:00:00,Center Pivot,2016/12/31 00:00:00,NaN,"Corn, Field",357722.315126,2289.389918,T19R29E29,Grant,2016.0,"{""type"":""Polygon"",""coordinates"":[[[-119.205228..."
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
last_row_id = double_dt.shape[0] - 1
double_dt = double_dt.drop([last_row_id], axis=0)
double_dt = double_dt.astype({"year": int})
double_dt.tail(2)

,Acres,CovrCrp,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,Notes,RtCrpTy,Shap_Ar,Shp_Lng,TRS,county,year,geo
7,59.0,NaN,Other,Unknown,NaN,58.999775,2003/07/01 00:00:00,Center Pivot,2016/12/31 00:00:00,NaN,"Corn, Field",238763.617994,2114.872186,T17R29E35,Grant,2016,"{""type"":""Polygon"",""coordinates"":[[[-119.141463..."
8,88.0,NaN,Hay/Silage,Alfalfa Hay,NaN,88.395109,2003/07/01 00:00:00,Center Pivot,2016/12/31 00:00:00,NaN,"Corn, Field",357722.315126,2289.389918,T19R29E29,Grant,2016,"{""type"":""Polygon"",""coordinates"":[[[-119.205228..."


# Compute acreage of each double peaked cultivar

In [36]:
acreage_per_cultivar_all = double_dt.groupby(["county", "year", "CropTyp"]).ExctAcr.sum().reset_index()

# Filter the double-peaked cultivars 
by those that actually might be double-cropped. In other words, toss orchards and those that cannot be doubled-cropped

In [53]:
double_crop_poten = double_dt[double_dt.CropTyp.isin(double_crop_potential_plants['Crop_Type'])]

# Compute acreage of potential double-cropped from double-peaked 
This is a subset of above table already

In [57]:
acr_per_potential_doubles = double_crop_poten.groupby(["county", "year", "CropTyp"]).ExctAcr.sum().reset_index()
acr_per_potential_doubles.head(2)

,county,year,CropTyp,ExctAcr
0,Grant,2016,"Pea, Green",87.664966


# Save the data

In [60]:
# Saving path
out_dir = data_dir + "/acreage_tables/"
os.makedirs(out_dir, exist_ok=True)

In [63]:
all_acr_path_name = out_dir + "all_cult_acr_LA_" + str(look_ahead) + "_df_"  + str(freedom_dg) + ".csv"

potential_double_acr_path_name = out_dir + "potential_cult_acr_LA_" + \
                                 str(look_ahead) + "_df_"  + str(freedom_dg) + ".csv"

In [64]:
acreage_per_cultivar_all.to_csv(all_acr_path_name, index = False)
acr_per_potential_doubles.to_csv(potential_double_acr_path_name, index = False)